In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm, boxcox
from scipy import stats
# from pandas_profiling import ProfileReport
import plotly.express as px
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans
import scipy.cluster.hierarchy as sch
import warnings
warnings.simplefilter(action='ignore', category=Warning)

In [ ]:
df= pd.read_csv('/kaggle/input/customer-segmentation-tutorial-in-python/Mall_Customers.csv')
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

In [ ]:
df.isna().sum()

In [ ]:
df1= df.iloc[:,2:5]
df1

In [ ]:
df1.corr()

In [ ]:
plt.figure(figsize=(10, 7))
matrix = np.triu(df1.corr())
sns.heatmap(df1.corr(), annot=True,linewidth=.8, mask=matrix, cmap="rocket");

In [ ]:
print(df.dtypes)


In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(15, 6))
sns.countplot(data=df, x='Gender', ax=axes[0])
axes[0].set_title('Countplot of Gender')
df['Gender'].value_counts().plot.pie(autopct='%1.1f%%', ax=axes[1])
axes[1].set_title('Pie Chart of Gender')
plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(18, 6))
sns.countplot(data=df, x='Annual Income (k$)', ax=axes[0])
axes[0].set_title('Countplot of Annual Income (k$)')
sns.countplot(data=df, x='Age', ax=axes[1])
axes[1].set_title('Countplot of Age')
plt.tight_layout()
plt.show()

In [ ]:
sns.distplot(df['Spending Score (1-100)'])

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(18, 6))
sns.boxplot(df['Spending Score (1-100)'], ax=axes[0])
axes[0].set_title('Spending Score (1-100)')
sns.boxplot(df['Annual Income (k$)'],ax=axes[1])
axes[1].set_title(' Annual Income (k$)')
sns.boxplot(df['Age'],ax=axes[2])
axes[2].set_title('Age')
plt.tight_layout()
plt.show()

In [ ]:
sns.pairplot(df, hue='Gender')

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
df = pd.read_csv('/kaggle/input/customer-segmentation-tutorial-in-python/Mall_Customers.csv')
X = df.drop(columns=['CustomerID', 'Age'], axis=1).values

In [ ]:
plt.figure(figsize=(15, 8))
sns.scatterplot(x='Annual Income (k$)', y='Spending Score (1-100)', hue='Gender', data=df, s=100, alpha=0.7)

plt.xlabel('Annual Income (k$)')
plt.ylabel('Spending Score (1-100)')
plt.title('Scatter Plot of Annual Income vs Spending Score')
plt.show()

In [ ]:
X = df.drop(columns=['CustomerID', 'Age', 'Gender'], axis=1).values

In [ ]:
wsse = []
for k in range(1, 11):
    kmeans = KMeans(n_clusters=k, init='k-means++',random_state=2)
    kmeans.fit(X)
    wsse.append(kmeans.inertia_)

# Plot the elbow method
plt.figure(figsize=(15, 8))
plt.plot(range(1, 11), wsse)
plt.title('Elbow Method for Optimal k')
plt.xlabel('Number of Clusters (k)')
plt.ylabel('Sum of Squared Distances (WSSE)')
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=5, init='k-means++', random_state=0)
df['Cluster'] = kmeans.fit_predict(X)

In [ ]:
kmeans.cluster_centers_

In [ ]:
plt.figure(figsize=(15, 8))
# Plot cluster centers
cluster_centers = kmeans.cluster_centers_
plt.scatter(cluster_centers[:, 0], cluster_centers[:, 1], c='black', marker='o', s=200, label='Cluster Centers')

# Plot clustered points with different colors
sns.scatterplot(x=X[:, 0], y=X[:, 1], hue=df['Cluster'], palette='viridis', s=100)
plt.legend(title='Clusters', loc='upper right')
plt.show()


In [ ]:
from sklearn.cluster import DBSCAN

In [ ]:
dbscan = DBSCAN(eps=9, min_samples=3)

In [ ]:
clusters = dbscan.fit_predict(X)

In [ ]:
np.unique(clusters)

In [ ]:
label=dbscan.labels_

In [ ]:
plt.figure(figsize=(15, 8))
plt.scatter(X[:, 0],X[:, 1], c=clusters, cmap='viridis', marker='o')
plt.title('DBSCAN Clustering')
plt.xlabel('Annual Income (k$)')
plt.ylabel('Spending Score (1-100)')
plt.legend(title='Clusters (DBSCAN)')
plt.show()

In [ ]:
from sklearn.cluster import DBSCAN

dbscan = DBSCAN(eps=9, min_samples=3)
clusters = dbscan.fit_predict(X)

unique_labels = np.unique(clusters)

plt.figure(figsize=(15, 8))
for label in unique_labels:
    if label == -1:
        # Plot noise points in black
        plt.scatter(X[clusters == label, 0], X[clusters == label, 1], c='black', marker='o', label=f'Noise ({label})')
    else:
        plt.scatter(X[clusters == label, 0], X[clusters == label, 1], label=f'Cluster {label}')

plt.title('DBSCAN Clustering')
plt.xlabel('Annual Income (k$)')
plt.ylabel('Spending Score (1-100)')
plt.legend(title='Clusters (DBSCAN)', loc='upper right')
plt.show()


In [ ]:
from sklearn import metrics

n_clusters_ = len(set(clusters)) - (1 if -1 in clusters else 0)  # Exclude noise points
print("Estimated number of clusters:", n_clusters_)

# 2) Estimated number of noise points
n_noise_ = list(clusters).count(-1)
print("Estimated number of noise points:", n_noise_)

# 3) Silhouette coefficient (not applicable if the number of clusters is 1 or -1)
if n_clusters_ > 1 or n_noise_ > 0:
    silhouette_coefficient = metrics.silhouette_score(X, clusters)
    print("Silhouette Coefficient:", silhouette_coefficient)
else:
    print("Silhouette Coefficient not applicable for 1 cluster or noise only.")


In [ ]:
import scipy.cluster.hierarchy as sch

plt.figure(figsize=(10,9))
plt.title('Customer Dendogram')
dend = sch.dendrogram(sch.linkage(X,method='ward'))

In [ ]:
from sklearn.cluster import AgglomerativeClustering 
cluster = AgglomerativeClustering(n_clusters=5, affinity="euclidean",linkage="ward")
labels_= cluster.fit_predict(X)

In [ ]:
labels_

In [ ]:
plt.figure(figsize=(10,7))
plt.scatter(X[:,0], X[:,1], c=cluster.labels_, cmap='rainbow')